# Getting Familiar with PuLP

In [1]:
# Import PuLP modeler functions
from pulp import *

In [2]:
# Initialize class
prob = LpProblem("LPexample", LpMaximize)

In [3]:
# Define variables
x = LpVariable('x', lowBound=30, cat='Continuous')
y = LpVariable('y', None, cat='Continuous')

In [4]:
# Define Objective Function
prob += 5*x + 7*y, "Z"

In [5]:
# Define Constraints
prob += 4*y <= 10 + x
prob += 2*y + 3*x <= 200

In [6]:
# Check the newly created problem
prob

LPexample:
MAXIMIZE
5*x + 7*y + 0
SUBJECT TO
_C1: - x + 4 y <= 10

_C2: 3 x + 2 y <= 200

VARIABLES
30 <= x Continuous
y free Continuous

In [7]:
# The problem is solved using PuLP's choice of Solver
prob.solve()
# The status of the solution 
LpStatus[prob.status]

'Optimal'

In [8]:
# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    print(v.name, "=", v.varValue)

x = 55.714286
y = 16.428571


In [9]:
# Print the optimized solution
print("The maximized value can be =", value(prob.objective))

The maximized value can be = 393.571427


**Exercise 1:** You are managing a small warehouse with a total effective storage size of 900 cubic meters. You have the option to sell 8 different products online, A, B, C, D, E, F, G, and H, each with unique size and profit as is detailed in the table below. Your task is to maximize your profit while considering what to stock and how many to stock for each item (note: ignore the demand for each item).

Conditions: 
- Product A and B  are similar with minor variations, you should at least store either item A or B to satisfy your long-term customers
- Product G and H are sold in pairs, i.e. if Product G is selected, then product H must also be selected.
            


|Item|Size | Profit|
| --- | --- |--- |
|A    |3      |240|
|B    |3      |245|
|C    |4      |250|
|D    |5      |410|
|E    |3.5    |300|
|F    |4      |150|
|G    |2      |140|
|H    |1      |100|

In [10]:
# Create a list of items
item = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']

# Create a dictionary of the size of each item
size = {'A':3, 'B':3, 'C':4,'D':5, 'E':3.5, 'F':4, 'G':2, 'H':1}

# Creat a dictionary of the profit of each item
prof = {'A':240, 'B':245, 'C':250,'D':410, 
        'E':300, 'F':150, 'G':140, 'H':100}

In [11]:
# Initialize the problem model 
model = LpProblem("warehousing", LpMaximize)

In [12]:
# Define Decision Variables
x = LpVariable.dicts('stock_', item, lowBound=0, cat='Integer')
x

{'A': stock__A,
 'B': stock__B,
 'C': stock__C,
 'D': stock__D,
 'E': stock__E,
 'F': stock__F,
 'G': stock__G,
 'H': stock__H}

In [13]:
# Define Objective Function
model += lpSum([prof[i]*x[i] for i in item])

In [14]:
# Define Constraints
model += lpSum([size[i]*x[i] for i in item]) <= 900,'Capacity Cons'
model += x['A'] + x['B'] >= 1, 'A or B'
model += x['G'] - x['H'] == 0, 'G=H'

In [15]:
# Solve Model
model.solve()

for i in item:
    print("{} stock {}".format(i, x[i].varValue))

print("The maxmized profit could be £{}".\
                format(value(model.objective)))

A stock 0.0
B stock 2.0
C stock 0.0
D stock 1.0
E stock 254.0
F stock 0.0
G stock 0.0
H stock 0.0
The maxmized profit could be £77100.0


**Exercise 2:** Assume the case study data is from a pizza store optimizing its logistics network across five regions (R1, R2, R3, R4 and R5). The pizza store is considering openning additional stores to cover the demands in these five regions. You are given the daily demand in each region, daily product capacity (small store or large store) of each store, and the delivery cost and fixed costs of building each store(thousands of GBP£).  The ship_cost shows the costs of producing in location i shipping to location j. 

In [16]:
import pandas as pd
Data = pd.read_csv("PizzaStoreLocation.csv", index_col='Region')
Data

,Demand,Low_fix,High_fix,Low_cap,High_cap,R1,R2,R3,R4,R5
Region,,,,,,,,,,
R1,5000,800,1200,2000,6000,5,12,17,8,7
R2,1800,1250,1650,2000,6000,12,4,10,7,8
R3,800,900,1450,2000,6000,17,10,3,11,8
R4,630,650,800,2000,6000,8,7,11,7,18
R5,490,666,950,2000,6000,7,8,10,18,6


In [17]:
fix_cost = Data[['Low_fix', 'High_fix']]
fix_cost.columns=['Low_cap','High_cap']
capacity = Data[['Low_cap', 'High_cap']]
ship_cost = Data[['R1', 'R2', 'R3','R4', 'R5']]
fix_cost

,Low_cap,High_cap
Region,,
R1,800,1200
R2,1250,1650
R3,900,1450
R4,650,800
R5,666,950


In [18]:
# Rename columns name for later quick access
fix_cost.columns=['Low_cap','High_cap']
fix_cost

,Low_cap,High_cap
Region,,
R1,800,1200
R2,1250,1650
R3,900,1450
R4,650,800
R5,666,950


In [19]:
capacity = Data[['Low_cap', 'High_cap']]
capacity

,Low_cap,High_cap
Region,,
R1,2000,6000
R2,2000,6000
R3,2000,6000
R4,2000,6000
R5,2000,6000


In [20]:
ship_cost = Data[['R1', 'R2', 'R3','R4', 'R5']]
ship_cost

,R1,R2,R3,R4,R5
Region,,,,,
R1,5,12,17,8,7
R2,12,4,10,7,8
R3,17,10,3,11,8
R4,8,7,11,7,18
R5,7,8,10,18,6


In [21]:
ship_cost.loc['R2','R4']

7

In [22]:
# Initialize model
model = LpProblem("Pizza_Store_Network_Design", LpMinimize)
loc = ['R1', 'R2', 'R3', 'R4', 'R5']
size = ['Low_cap','High_cap']

# Define variables
x = LpVariable.dicts("quantity_", [(i,j) for i in loc for j in loc],
                     lowBound=0, upBound=None, cat='Integer')
y = LpVariable.dicts("store_",[(i,s) for s in size for i in loc],
                     cat='Binary')

# Define objective function
model += (lpSum([fix_cost.loc[i,s] * y[(i,s)] 
                 for s in size for i in loc])
          + lpSum([ship_cost.loc[i,j] * x[(i,j)] 
                   for i in loc for j in loc]))

# Define the constraints
for j in loc:
    model += lpSum([x[(i, j)] for i in loc]) == Data.loc[j,'Demand']
for i in loc:
    model += lpSum([x[(i, j)] for j in loc]) <= \
                  lpSum([capacity.loc[i,s] * y[(i,s)] for s in size])
    
# Solve
model.solve()
# Print model status
print(LpStatus[model.status])

# Print Solved Store Delivery Options
Sol = [{'Delivery':"{} to {}".format(i,j), 'Amount':x[(i,j)].varValue}
for i in loc for j in loc]
print(pd.DataFrame(Sol))

# Print Solved Store Openning Options
Sol = [{'loc':i, 'Low_cap':y[(i,size[0])].varValue, 
        'High_cap':y[(i,size[1])].varValue} for i in loc]
print(pd.DataFrame(Sol))

# Print Total Optimized Costs
print("Total Optimized Costs = £{}". format(value(model.objective)))


Optimal
    Delivery  Amount
0   R1 to R1  5000.0
1   R1 to R2     0.0
2   R1 to R3     0.0
3   R1 to R4     0.0
4   R1 to R5   490.0
5   R2 to R1     0.0
6   R2 to R2  1800.0
7   R2 to R3     0.0
8   R2 to R4   630.0
9   R2 to R5     0.0
10  R3 to R1     0.0
11  R3 to R2     0.0
12  R3 to R3   800.0
13  R3 to R4     0.0
14  R3 to R5     0.0
15  R4 to R1     0.0
16  R4 to R2     0.0
17  R4 to R3     0.0
18  R4 to R4     0.0
19  R4 to R5     0.0
20  R5 to R1     0.0
21  R5 to R2     0.0
22  R5 to R3     0.0
23  R5 to R4     0.0
24  R5 to R5     0.0
  loc  Low_cap  High_cap
0  R1      0.0       1.0
1  R2      0.0       1.0
2  R3      1.0       0.0
3  R4      0.0       0.0
4  R5      0.0       0.0
Total Optimized Costs = £46190.0
